In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Latin_Macro_Watch_Dataset.csv")

<ipython-input-2-e9095707c131>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Latin_Macro_Watch_Dataset.csv")


In [60]:
variables = df.groupby(["Frequency", "Indicator", "Unit", "Definition"]).count().reset_index()

array(['Annual', 'Quarterly', 'Monthly'], dtype=object)

In [48]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year

In [67]:
writer = pd.ExcelWriter('LMV_RAW.xlsx')

for freq in df.Frequency.unique():
    anual = df[df.Frequency==freq]

    count_df = anual.groupby(["Country", "year"])["Value"].count().reset_index()
    count_pv = count_df.pivot_table(index='Country', columns='year', values='Value', fill_value=0)

    nan_df = anual.groupby(["Country", "year"])["Value"].apply(lambda x: x.isna().sum()).reset_index()
    nan_pv = nan_df.pivot_table(index='Country', columns='year', values='Value', fill_value=0)

    total_pv = nan_pv + count_pv
    nan_pvp = nan_pv/count_pv
    porcentaje_nan = ((nan_pv/total_pv)*100)

    total_pv["N"] = "Total"
    nan_pv["N"] = "Nans"
    porcentaje_nan["N"] = "%Nans"

    dff = pd.concat([total_pv, nan_pv, porcentaje_nan], axis = 0)

    dff.to_excel(writer, sheet_name=f'{freq}')



<ipython-input-67-1571f9846545>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  writer = pd.ExcelWriter('LMV_RAW.xlsx')


In [76]:
variables.to_excel(writer, sheet_name='variable_frequency')
writer.close()

In [4]:
# valores = freq_df.drop_duplicates(["indicator", "unit", "definition"])[["area","topic","source", "indicator", "definition", "unit"]].reset_index(drop=True).sort_values(["indicator", "unit"])
# valores.to_excel("valores_unicos.xlsx")

In [8]:
# valores = df.drop_duplicates(["indicator", "unit", "definition"])[["area","topic","source", "indicator", "definition", "unit"]].reset_index(drop=True).sort_values(["indicator", "unit", "definition"])
# valores.to_excel("valores_unicos_third.xlsx")

<ipython-input-8-c9debf40878e>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  valores.to_excel("valores_unicos_third.xlsx")


In [162]:
dff = pd.DataFrame()
for variable, unidad in zip(valores.indicator, valores.unit):
    dfk = df[(df.indicator == variable) & (df.unit == unidad)]
    dfk['value'] = dfk['value'].str.replace(',', '').astype(float)

    dfv = dfk.groupby("country")["value"].mean().reset_index()
    dfv.columns = ["country", variable]

    stats = {
        'min': dfk['value'].min(),
        'max': dfk['value'].max(),
        'mean': dfk['value'].mean(),
        'p50': dfk['value'].quantile(0.50),
        'std': dfk['value'].std(),
        'nans': dfk['value'].isna().sum(),
        'N': len(dfk['value'])
    }

    # Convertir las estadísticas en un DataFrame temporal
    stats_df = pd.DataFrame(stats, index=['stats'])

    stats_df = stats_df.T.reset_index()
    stats_df.columns = ["country", variable]

    # Concatenar el DataFrame original con el nuevo DataFrame de estadísticas
    dfv = pd.concat([dfv, stats_df])
    dfv.index = dfv.country
    dfv = dfv[[variable]]

    dff = pd.concat([dff, dfv], axis = 1)

# dff.to_excel("variable_desc.xlsx")

<ipython-input-162-f2dfb7d953be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk['value'] = dfk['value'].str.replace(',', '').astype(float)
<ipython-input-162-f2dfb7d953be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk['value'] = dfk['value'].str.replace(',', '').astype(float)
<ipython-input-162-f2dfb7d953be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: